In [ ]:
#!pip install jupyter_contrib_nbextensions
#!jupyter contrib nbextension install --user
#!jupyter nbextension enable python-markdown/main

In [53]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Fri Oct 31 15:44:16 2025

@author: hmair
"""

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

file = pd.read_csv("/content/drive/Shareddrives/GIO-GINA/Protocol Review and Updates/Transparency Secchi Disk/Secchi.csv")

/tmp/ipython-input-4270503554.py:13: DtypeWarning: Columns (9,15,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50) have mixed types. Specify dtype option on import or set low_memory=False.
  file = pd.read_csv("/content/drive/Shareddrives/GIO-GINA/Protocol Review and Updates/Transparency Secchi Disk/Secchi.csv")


In [42]:
print('num fresh observations', len(file[file['water_body_type'] == 'fresh_water']))
print('num salt observations', len(file[file['water_body_type'] == 'salt_water']))
print('num brackish observations', len(file[file['water_body_type'] == 'brackish']))

num fresh observations 48408
num salt observations 8609
num brackish observations 3318


### Negative Depths

Disk disappearance depth was calculated using the following equation:

```
((disappearance - distance_to_surface) + (reappearance - distance_to_surface))/2
```

This can produce negative results in two ways:
1. The distance to surface is greater than the depth to disappearance. This happens if the observer takes a measurement from high above the water surface, such as on a dock or bridge, and the water has low transparency (reliable case).
2. If the observer did not include the height above water in their disappearance depth total, and only entered the depth to disappearance *below the water surface* (not reliable).


In [43]:
negatives = file[file['transparency_disk_image_disappearance_m'] < 0]
print("total observations", len(file))
print("total number of negative depths", len(negatives))
print("fresh water negatives", len(negatives[negatives['water_body_type'] == "fresh_water"]))
print("salt water negatives", len(negatives[negatives['water_body_type'] == "salt_water"]))
print("brackish water negatives", len(negatives[negatives['water_body_type'] == "brackish"]))
print("unknown water negatives", len(negatives[negatives['water_body_type'] == "unknown"]))

total observations 62008
total number of negative depths 2308
fresh water negatives 1783
salt water negatives 231
brackish water negatives 187
unknown water negatives 107


In [52]:
##null vlaues?
fresh = file[file['water_body_type'] == 'fresh_water']
print("fresh null", fresh['transparency_disk_image_disappearance_m'].isnull().sum())

salt = file[file['water_body_type'] == 'salt_water']
print("salt null", salt['transparency_disk_image_disappearance_m'].isnull().sum())

brackish = file[file['water_body_type'] == 'brackish']
print("brackish null", brackish['transparency_disk_image_disappearance_m'].isnull().sum())

unknown = file[file['water_body_type'] == 'unknown']
print("unknown null", unknown['transparency_disk_image_disappearance_m'].isnull().sum())


fresh null 28
fresh null distance 14282
salt null 1
brackish null 4
unknown null 3


In [63]:
positives = file[file['transparency_disk_image_disappearance_m'] >= 0]
positives['sample_distance_to_surface_m'] = positives['sample_distance_to_surface_m'].fillna(0)
print("Number of positive values", len(positives))

print("Mean positive depth", positives['transparency_disk_image_disappearance_m'].mean())
print("STD of positive depth", positives['transparency_disk_image_disappearance_m'].std())
print("Range of positive values", positives['transparency_disk_image_disappearance_m'].min(), positives['transparency_disk_image_disappearance_m'].max())

Number of positive values 59662
Mean positive depth 1.7542052194026347
STD of positive depth 4.870708155872528
Range of positive values 0.0 45.0


/tmp/ipython-input-2672642163.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  positives['sample_distance_to_surface_m'] = positives['sample_distance_to_surface_m'].fillna(0)


In [ ]:
print("Mean depth", file['transparency_disk_image_disappearance_m'].mean())
print("STD of depth", file['transparency_disk_image_disappearance_m'].std())
print("Range of values", file['transparency_disk_image_disappearance_m'].min(), file['transparency_disk_image_disappearance_m'].max())

Mean depth 1.6345166741971924
STD of depth 4.865621097562805
Range of values -49.0 45.0


In [ ]:
print("Mean negative depth", negatives['transparency_disk_image_disappearance_m'].mean())
print("STD of negative depth", negatives['transparency_disk_image_disappearance_m'].std())
print("Range of negative values", negatives['transparency_disk_image_disappearance_m'].min(), negatives['transparency_disk_image_disappearance_m'].max())

Mean negative depth -1.4594425909878683
STD of negative depth 3.529370445776955
Range of negative values -49.0 -1e-16


# Positive Values

In [64]:
highdist = file[file['sample_distance_to_surface_m'] > 5]

fresh_highdist = highdist[highdist['water_body_type'] == 'fresh_water']
print("distance above fresh water > 5m:", len(fresh_highdist))
low_dist = positives[(positives['sample_distance_to_surface_m'] <= 5)]
fresh_clean = low_dist[low_dist['water_body_type'] == 'fresh_water']

salt_highdist = highdist[highdist['water_body_type'] == 'salt_water']
print("distance above salt water > 5m:", len(salt_highdist))
salt_clean = low_dist[low_dist['water_body_type'] == 'salt_water']

brack_highdist = highdist[highdist['water_body_type'] == 'brackish']
print("distance above brackish water > 5m:", len(brack_highdist))
brack_clean = low_dist[low_dist['water_body_type'] == 'brackish']

distance above fresh water > 5m: 3648
distance above salt water > 5m: 703
distance above water > 5m: 552


In [65]:
##check for 0 lat/lon

badlocs = file[(file['latitude'] == 0) & (file['longitude'] == 0)]
print("fresh bad locations", len(badlocs[badlocs['water_body_type'] == 'fresh_water']))
print("salt bad locations", len(badlocs[badlocs['water_body_type'] == 'salt_water']))
print("brackish bad locations", len(badlocs[badlocs['water_body_type'] == 'brackish']))

fresh bad locations 485
salt bad locations 0
brackish bad locations 0


In [66]:
#disappearance/reappearance depth is same as distance above water

condition = (file['sample_distance_to_surface_m'] == file['sample_transparency_disk_image_disappearance_m']) | (file['sample_distance_to_surface_m'] == file['sample_transparency_disk_image_reappearance_m'])
samedist = file[condition]
fresh_samedist = samedist[samedist['water_body_type'] == 'fresh_water']
condition1 = (fresh_clean['sample_distance_to_surface_m'] == fresh_clean['sample_transparency_disk_image_disappearance_m']) | (fresh_clean['sample_distance_to_surface_m'] == fresh_clean['sample_transparency_disk_image_reappearance_m'])
print("fresh same distance", len(fresh_samedist))
fresh_clean = fresh_clean[~condition1]

salt_samedist = samedist[samedist['water_body_type'] == 'salt_water']
condition2 = (salt_clean['sample_distance_to_surface_m'] == salt_clean['sample_transparency_disk_image_disappearance_m']) | (salt_clean['sample_distance_to_surface_m'] == salt_clean['sample_transparency_disk_image_reappearance_m'])
print("salt same distance", len(salt_samedist))
salt_clean = salt_clean[~condition2]

brack_samedist = samedist[samedist['water_body_type'] == 'brackish']
condition3 = (brack_clean['sample_distance_to_surface_m'] == brack_clean['sample_transparency_disk_image_disappearance_m']) | (brack_clean['sample_distance_to_surface_m'] == brack_clean['sample_transparency_disk_image_reappearance_m'])
print("brackish same distance", len(brack_samedist))
brack_clean = brack_clean[~condition3]

fresh same distance 2500
salt same distance 330
brackish same distance 51


In [70]:
##depth differences
fresh_clean['depth_difference'] = abs(fresh_clean['sample_transparency_disk_image_disappearance_m'] - fresh_clean['sample_transparency_disk_image_reappearance_m'])
print('range of differences', fresh_clean['depth_difference'].min(), fresh_clean['depth_difference'].max())

salt_clean['depth_difference'] = abs(salt_clean['sample_transparency_disk_image_disappearance_m'] - salt_clean['sample_transparency_disk_image_reappearance_m'])
print('range of differences', salt_clean['depth_difference'].min(), salt_clean['depth_difference'].max())

brack_clean['depth_difference'] = abs(brack_clean['sample_transparency_disk_image_disappearance_m'] - brack_clean['sample_transparency_disk_image_reappearance_m'])
print('range of differences', brack_clean['depth_difference'].min(), brack_clean['depth_difference'].max())

fresh_clean.to_csv("/content/drive/Shareddrives/GIO-GINA/Protocol Review and Updates/Transparency Secchi Disk/GLOBE_secchi_transparency_freshwater.csv")
salt_clean.to_csv("/content/drive/Shareddrives/GIO-GINA/Protocol Review and Updates/Transparency Secchi Disk/GLOBE_secchi_transparency_saltwater.csv")
brack_clean.to_csv("/content/drive/Shareddrives/GIO-GINA/Protocol Review and Updates/Transparency Secchi Disk/GLOBE_secchi_transparency_brackish.csv")

print('final num fresh', len(fresh_clean))
print('final num salt', len(salt_clean))
print('final num brackish', len(brack_clean))

range of differences 0.0 29.68
range of differences 0.0 11.0
range of differences 0.0 10.0
final num fresh 40659
final num salt 7392
final num brackish 2492


In [68]:
#Count number of observations with 1, 2, or 3 samples
counts_per_id = fresh_clean.groupby('transparency_id')['sample_number'].count()
final_result = counts_per_id.value_counts().sort_index()

print("fresh", final_result)

counts_per_id = salt_clean.groupby('transparency_id')['sample_number'].count()
final_result2 = counts_per_id.value_counts().sort_index()

print("salt", final_result2)

counts_per_id = brack_clean.groupby('transparency_id')['sample_number'].count()
final_result3 = counts_per_id.value_counts().sort_index()

print("brackish", final_result3)


fresh sample_number
1     6705
2      711
3    10844
Name: count, dtype: int64
salt sample_number
1    5077
2      97
3     707
Name: count, dtype: int64
brackish sample_number
1    922
2     14
3    514
Name: count, dtype: int64


In [69]:
#check that the totals match
print(sum(final_result))
print(fresh_clean['transparency_id'].nunique())

print(sum(final_result2))
print(salt_clean['transparency_id'].nunique())

print(sum(final_result3))
print(brack_clean['transparency_id'].nunique())

18260
18260
5881
5881
1450
1450
